In [ ]:
#importing required libraries
import tkinter as tk
from tkinter import *
from tkinter import filedialog
from keras.preprocessing.image import img_to_array
from tkinter.filedialog import askopenfile
from skimage.feature import graycomatrix,graycoprops
from PIL import Image, ImageTk
import skimage
import numpy as np
from keras import backend as K
import PIL
import cv2
import pickle
from keras.models import load_model
import matplotlib

#create window and add button,label
global bg
matplotlib.use('Agg')
my_w = tk.Tk()
my_w.geometry("1100x700")
my_w.title('brain stroke system')
my_w.configure(background="sky blue")
my_font1=('times', 18, 'bold')


l1 = tk.Label(my_w,text='Add brain image data',width=30,font=my_font1)  
l1.grid(row=1,column=1)
b1 = tk.Button(my_w, text='Upload File', width=20,command = lambda:upload_file())
b1.grid(row=2,column=1) 


#function for identifying type of stroke
def glcm_algo(img,img3):
    global temp
    size = (500, 500)
    img = img.resize(size, PIL.Image.ANTIALIAS)
    img3=cv2.resize(img3,size)
    img=img.convert('L')
    im = skimage.img_as_ubyte(img)
    im =im// 32
    glcm=skimage.feature.graycomatrix(im, [1], [0], levels=8)
    contrast=skimage.feature.graycoprops(glcm, 'contrast')[0][0]
    energy=skimage.feature.graycoprops(glcm, 'energy')[0][0]
    homogeneity=skimage.feature.graycoprops(glcm, 'homogeneity')[0][0]
    correlation=skimage.feature.graycoprops(glcm, 'correlation')[0][0]
    dissimilarity=skimage.feature.graycoprops(glcm, 'dissimilarity')[0][0]
    ASM=skimage.feature.graycoprops(glcm, 'ASM')[0][0]
    
    loaded_model = pickle.load(open("model_feature", 'rb'))
    result=loaded_model.predict([[contrast,energy,homogeneity,correlation,dissimilarity,ASM]])
    if(result==0):
        l4 = tk.Label(my_w,text='Damage Status: Detected\n Type:Ischemic',width=30,font=my_font1,fg="red")
        l4.grid(row=2,column=2,padx=10,pady=10)
        for i in range(0,500):
            for j in range(0,500):
                if(img3[i][j][0]>=59 and img3[i][j][0]<=75 and img3[i][j][1]>=59 and img3[i][j][1]<=75 and img3[i][j][2]>=59 and img3[i][j][2]<=75):
                    img3[i][j]=[255,0,0]
        im=Image.fromarray(img3)
        temp=ImageTk.PhotoImage(im)
        b4 =tk.Button(my_w,image=temp)
        b4.grid(row=3,column=2,padx=10,pady=10)
        
        
        
    else:
        l5= tk.Label(my_w,text='Damage Status: Detected\n Type:Heammorhagic',width=30,font=my_font1,fg="red")
        l5.grid(row=2,column=2,padx=10,pady=10)
        for i in range(0,500):
            for j in range(0,500):
                if(img3[i][j][0]>=192 and img3[i][j][0]<=202 and img3[i][j][1]>=192 and img3[i][j][1]<=202 and img3[i][j][2]>=192 and img3[i][j][2]<=202):
                    img3[i][j]=[255,0,0]
        im=Image.fromarray(img3)
        temp=ImageTk.PhotoImage(im)
        b3 =tk.Button(my_w,image=temp)
        b3.grid(row=3,column=2,padx=10,pady=10)
        
    
         
#function for uploading image and displaying it on window.
def upload_file():
    global img
    global happy
    f_types = [('Jpg Files', '*.jpg')]
    filename = filedialog.askopenfilename(filetypes=f_types)
    img1=Image.open(filename)
    happy=Image.open("normal.gif")
    img2=cv2.imread(filename)
    img3=img2
    img = img1.resize((500,500), PIL.Image.ANTIALIAS)
    img = ImageTk.PhotoImage(img)
    b2 =tk.Button(my_w,image=img)
    b2.grid(row=3,column=1,padx=10,pady=10)
    size=(64,64)
    model1=load_model("damage_detection.h5")
    img2 = cv2.resize(img2,tuple((64,64)), PIL.Image.ANTIALIAS)
    img_array = img_to_array(img2)
    if K.image_data_format() == "channels_last":
        img_array = np.reshape(img_array,[1,64,64,3])
    else:
        img_array = np.reshape(img_array,[1,3,64,64])
    img_array = img_array/255
    prediction = model1.predict(img_array)
    predicted_class=np.argmax(prediction,axis=1)
    if(predicted_class==0):
        l6 = tk.Label(my_w,text='Damage Status: Not Detected\nType:Normal',width=30,font=my_font1,fg="green")
        l6.grid(row=2,column=2,padx=10,pady=10)
        happy = ImageTk.PhotoImage(happy)
        b5 =tk.Button(my_w,image=happy)
        b5.grid(row=3,column=2)
        
        
    else:
        glcm_algo(img1,img3)
        
        

my_w.mainloop() 

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\vishal\Miniconda3\envs\NB_2222\lib\site-packages\PIL\Image.py", line 2882, in open
    fp.seek(0)
AttributeError: 'str' object has no attribute 'seek'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\vishal\Miniconda3\envs\NB_2222\lib\tkinter\__init__.py", line 1883, in __call__
    return self.func(*args)
  File "<ipython-input-2-95757bb2552c>", line 30, in <lambda>
    b1 = tk.Button(my_w, text='Upload File', width=20,command = lambda:upload_file())
  File "<ipython-input-2-95757bb2552c>", line 87, in upload_file
    img1=Image.open(filename)
  File "C:\Users\vishal\Miniconda3\envs\NB_2222\lib\site-packages\PIL\Image.py", line 2884, in open
    fp = io.BytesIO(fp.read())
AttributeError: 'str' object has no attribute 'read'
C:\Users\vishal\Miniconda3\envs\NB_2222\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle